##Define Tree Stucture


In [5]:
# tree.py
import numpy as np
import random
import copy
from collections import deque
import bisect


class TreeNode:
    def __init__(self,val=0,height=1,ismut=False,left=None,right=None):
        self.val=val
        '''
        this stores the value of the TreeNode.
        For operator it will be 0(AND),1(OR),2(NOT).
        For filter values it can take values from 0-15
        '''
        self.left=left#denotes the left child of the node
        self.right=right#denotes the right child of the node
        self.height=height#denotes the height of the tree till this node
        self.ismut=ismut#Stores whether it's current state is mutated or not
    def __repr__(self):
            return f"TreeNode({self.val})"

def bfs(root,l1):
        '''
        The BFS function is to search the node at a particular depth(l1) and at a
        particular horizontal coordinate of the tree(which we select randomly).
        This is done in lieu of crossover and mutation which we define later
        (Basically at a particular depth which node number to choose.)
        '''
        # if self.root is None:
        #     return None
        if l1 == 0:
            return root
        queue=deque([root])
        while (queue and l1):
            x=len(queue)
            while(x):
                node=queue.popleft()
                if(node.left):
                    queue.append(node.left)
                if(node.right):
                    queue.append(node.right)
                x-=1
            l1-=1
        l=len(queue)
        if(l==0):
          return None
        ind=np.random.randint(0,l)
        while(ind):
            queue.popleft()
            ind-=1
        return queue[0]

def mutation(root):
        '''
        If a tree is to undergo mutation we select a random node where we need to do
        the mutation using bfs and set it's mutate to not of the previous value.
        '''
        l=root.height
        r=np.random.randint(0,l)
        node=bfs(root,r)
        if(node.height==1):
          node.val=np.random.randint(0,16)
        else:
          if(node.val==1):
            node.val=0
          else:
            node.val=1
        node.ismut=not node.ismut



##Create the Tree


In [91]:
'''
The create_tree functions are a set of over loaded functions defined to create a new tree
using given children trees and operator nodes.The first one is for binary operator.
The second one is for unary operator
'''
def unary_create_tree(tree1,op_node):
  #With NOT operator
  op_node.left=tree1
  op_node.height=tree1.height+1
  return op_node

In [92]:
def binary_create_tree(tree1,tree2,op_node):
  #With AND-OR operator
  r=np.random.randint(0,2)
  if(r):
    op_node.left=tree1
    op_node.right=tree2
    op_node.height=tree1.height+1
  else:
    op_node.left=tree2
    op_node.right=tree1
    op_node.height=tree1.height+1
  return op_node

##Calculate Fitness


In [93]:
'''
The signal tree fitness function is to compute the fitness of a signal tree.We take the base filters fitness in the base_fit_arr nd the tree
whose fitness we ought to calculate. But the buy or sell signal tree is sent individually to each of these functions to get the fitness signal on
the entire dataset.
'''
def signal_tree_fitness(base_fit_arr,tree):
  if(tree==None):
    return np.zeros(len(base_fit_arr[0]))
  if(tree.left==None and tree.right==None):
    return base_fit_arr[tree.val]
  child_fit_tot=np.zeros(len(base_fit_arr[0]))
  tree_fit_tot=np.zeros(len(base_fit_arr[0]))
  if(tree.val==2):

    child_fit_arr_left=signal_tree_fitness(base_fit_arr,tree.left)
    child_fit_tot=np.logical_not(child_fit_arr_left).astype(int)
    # print(tree.val,tree.height)
    # print(child_fit_tot)
  else:
    child_fit_arr_left=signal_tree_fitness(base_fit_arr,tree.left)
    child_fit_arr_right=signal_tree_fitness(base_fit_arr,tree.right)
    if(tree.val==1):
      child_fit_tot=np.logical_or(child_fit_arr_left,child_fit_arr_right).astype(int)
    else:
      child_fit_tot=np.logical_and(child_fit_arr_left,child_fit_arr_right).astype(int)
  return child_fit_tot

In [1]:
def chromosome_fitness_check(base_buy_arr,base_sell_arr,buy_tree,sell_tree,dataset,lot = 100000,cap = 100000,f=0):
    buy_signal=signal_tree_fitness(base_buy_arr,buy_tree)
    sell_signal=signal_tree_fitness(base_sell_arr,sell_tree)

    capital=[cap]
    buy_price=[]
    sell_price=[]
    for i in range(len(df)):
        if buy_signal[i]-sell_signal[i]==1:
            buy_price.append(dataset[i])
            sell_price.append(np.nan)
        elif buy_signal[i]-sell_signal[i]==-1:
            buy_price.append(np.nan)
            sell_price.append(dataset[i])
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)

    buyx=0
    sellx=0
    buypx=0
    sellpx=0
    buyp=0
    sellp=0
    #pos_type=[]
    for i in range(len(dataset)):
        if not np.isnan(buy_price[i]):
            if sellx==1:                       #closing sell position
                capital.append(capital[-1])
                sellx=0
                sellpx=0
                sellp=0
                #pos_type.append("close")
                cap=capital[-1]
            else:                              #opening buy position
                capital.append(capital[-1])
                buypx=buypx+1
                if buypx==1:
                    buyp=dataset[i]
                    cap=cap-(buyp*lot)
                    buyx=1
                elif buypx>1 :                           #to keep count of more than 1 positions
                    buyp=(buyp+dataset[i])
                    cap=cap-(buyp*lot/buypx)
                    buyx=1
                #pos_type.append("open")


        elif not np.isnan(sell_price[i]):
            if buyx==1:
                capital.append(capital[-1])
                buyx=0
                buypx=0
                buyp=0
                #pos_type.append("close")
                cap=capital[-1]
            else:
                capital.append(capital[-1])
                sellpx=sellpx+1
                if sellpx==1:
                    sellp=dataset[i]
                    cap=cap+(sellp*lot)
                    sellx=1
                elif sellpx>1:
                    sellp=(sellp+dataset[i])
                    cap=cap+(sellp*lot/sellpx)
                    sellx=1
                #pos_type.append("open")

        elif buyx==1:
            capital.append(cap+(dataset[i]*lot*buypx))
            #pos_type.append(np.nan)
        elif sellx==1:
            capital.append(cap-(dataset[i]*lot*sellpx))
            #pos_type.append(np.nan)
        else:
            capital.append(capital[-1])
            cap=capital[-1]
            #pos_type.append(np.nan)
    del capital[0]
    capital=np.array(capital,dtype=float)
    for i in range(len(dataset)):
        capital[i]=capital[i]-100000
    if(f):
      return capital
    mu = np.mean(capital)
    std = np.std(capital)
    if std == 0:
      return 0
    else:
      return mu/std

    # return capital[-1]
    # return sharpe

##Perform CrossOver and Move to Next Generation


In [95]:
def not_rootswap(root_a,root_b):
        x = root_a.left
        r = np.random.randint(0,2)
        if r:
            root_a.left = root_b.right
            root_b.right = x
        else:
            root_a.left = root_b.left
            root_b.left = x
def gen_rootswap(root_a,root_b):
    x=root_a
    root_a=root_b
    root_b=x

#This is the crossover between 2 buy or 2 sell signals at any random node at a particular depth d.
def crossover(tree1,tree2):
    h = tree1.height
    depth = np.random.randint(0,h-1) # range = [0 to h-2]
    child1 = copy.deepcopy(tree1)
    child2 = copy.deepcopy(tree2)
    root1 = bfs(child1,depth)
    root2 = bfs(child2,depth)

    # if both NOT operator
    if root1.val==2 and root2.val==2:
        x = root1.left
        root1.left = root2.left
        root2.left = x
    # if root1 is NOT operator
    elif root1.val == 2:
        not_rootswap(root1,root2)
    # if root2 is NOT operator
    elif root2.val == 2:
        not_rootswap(root1,root2)
    # if none of them are NOT operator
    else:
        swap_choice = random.choice(['left_left', 'right_right', 'left_right', 'right_left'])
        if swap_choice == 'left_left':
          gen_rootswap(root1.left,root2.left)
        if swap_choice == 'left_right':
          gen_rootswap(root1.left,root2.right)
        if swap_choice == 'right_left':
          gen_rootswap(root1.right,root2.left)
        if swap_choice == 'right_right':
          gen_rootswap(root1.right,root2.right)
    return child1,child2

In [96]:
from scipy.stats import pearsonr
def similarity(pnl_arr1,pnl_arr2):
  #print("pnl_arr1",pnl_arr1)
  #print("pnl_arr2",pnl_arr2)
  l=min(len(pnl_arr1),len(pnl_arr2))
  if(l<=1):
    return 0
  return pearsonr(pnl_arr1[:l],pnl_arr2[:l])[0]

In [97]:
def tournament_selection(fitness_arr, k = 3):
    tournament = random.sample(list(enumerate(fitness_arr)), k)
    winner = max(tournament, key=lambda x: x[1][0])
    id = winner[1][1]
    return id

def next_generation(base_buy,base_sell,buy_tree_pop,sell_tree_pop,dataset):
  fitness_arr = [(chromosome_fitness_check(base_buy, base_sell, buy_tree, sell_tree, dataset), index)
               for index, (buy_tree, sell_tree) in enumerate(zip(buy_tree_pop, sell_tree_pop))]
  fitness_pnl_arr=[(chromosome_fitness_check(base_buy, base_sell, buy_tree, sell_tree, dataset,100000,100000,1))
               for index, (buy_tree, sell_tree) in enumerate(zip(buy_tree_pop, sell_tree_pop))]
  threshold=0.75
  fitness_arr_new=fitness_arr
  for fit_val,index in fitness_arr:#code for shared fitness to penalize similar chromosomes
    fit_new=fit_val
    cnt=0
    for fit1,index1 in fitness_arr:
      if(index1!=index and similarity(fitness_pnl_arr[index],fitness_pnl_arr[index1])>threshold):
        cnt+=1
    fit_new=fit_new/(1.0+cnt)
    fitness_arr_new[index]=fit_new,index
  fitness_arr=fitness_arr_new
  sz = len(fitness_arr)
  fitness_arr = sorted(fitness_arr, key=lambda x: x[0], reverse = True) # sorts according to first key

  fit = np.array([t[0] for t in fitness_arr])
  print(len(fitness_arr),np.mean(fit[:20]))
  n=len(fitness_arr)//100 #hall of fame
  buy_tree_pop_new,sell_tree_pop_new=[],[]
  for i in range(n):
    id = fitness_arr[i][1]
    buy_tree_pop_new.append(buy_tree_pop[id])
    sell_tree_pop_new.append(sell_tree_pop[id])

  for i in range((sz-n)//2):
    id1 = tournament_selection(fitness_arr)
    id2 = tournament_selection(fitness_arr)
    bpar1 = buy_tree_pop[id1]
    spar1 = sell_tree_pop[id1]
    bpar2 = buy_tree_pop[id2]
    spar2 = sell_tree_pop[id2]

    if random.random() < 0.6:
      child1,child2=crossover(bpar1,bpar2)
      child11,child22=crossover(spar1,spar2)
      if(max(chromosome_fitness_check(base_buy,base_sell,child1,child11,dataset),chromosome_fitness_check(base_buy,base_sell,child2,child22,dataset))>
       min(chromosome_fitness_check(base_buy,base_sell,bpar1,spar1,dataset),
           chromosome_fitness_check(base_buy,base_sell,bpar2,spar2,dataset))):
         buy_tree_pop_new.append(child1)
         sell_tree_pop_new.append(child11)
         buy_tree_pop_new.append(child2)
         sell_tree_pop_new.append(child22)
      elif(max(chromosome_fitness_check(base_buy,base_sell,child1,child22,dataset),chromosome_fitness_check(base_buy,base_sell,child2,child11,dataset))>
          min(chromosome_fitness_check(base_buy,base_sell,bpar1,spar1,dataset),
            chromosome_fitness_check(base_buy,base_sell,bpar2,spar2,dataset))):
          buy_tree_pop_new.append(child1)
          sell_tree_pop_new.append(child22)
          buy_tree_pop_new.append(child2)
          sell_tree_pop_new.append(child11)
      else:
        buy_tree_pop_new.append(bpar1)
        sell_tree_pop_new.append(spar1)
        buy_tree_pop_new.append(bpar2)
        sell_tree_pop_new.append(spar2)
    else:
        buy_tree_pop_new.append(bpar1)
        sell_tree_pop_new.append(spar1)
        buy_tree_pop_new.append(bpar2)
        sell_tree_pop_new.append(spar2)

  n_mut_arr=np.random.randint(0,len(buy_tree_pop_new),size=len(buy_tree_pop_new)//10)
  for i in n_mut_arr:
    mutation(buy_tree_pop_new[i])
  n_mut_arr1=np.random.randint(0,len(buy_tree_pop_new),size=len(buy_tree_pop_new)//10)
  for i in n_mut_arr1:
    mutation(sell_tree_pop_new[i])

  fitness_new_arr = [(chromosome_fitness_check(base_buy, base_sell, buy_tree, sell_tree, dataset), index)
               for index, (buy_tree, sell_tree) in enumerate(zip(buy_tree_pop_new, sell_tree_pop_new))]
  fitness_new_pnl_arr=[(chromosome_fitness_check(base_buy, base_sell, buy_tree, sell_tree, dataset,100000,100000,1))
               for index, (buy_tree, sell_tree) in enumerate(zip(buy_tree_pop_new, sell_tree_pop_new))]
  fitness_arr_gen_new=fitness_new_arr
  for fit_val,index in fitness_new_arr:#code for shared fitness to penalize similar chromosomes
    fit_new=fit_val
    cnt=0
    for fit1,index1 in fitness_new_arr:
      if(index1!=index and similarity(fitness_new_pnl_arr[index],fitness_new_pnl_arr[index1])>threshold):
        cnt+=1
    fit_new=fit_new/(1.0+cnt)
    fitness_arr_gen_new[index]=fit_new,index
  fitness_new_arr=fitness_arr_gen_new
  fitness_new_arr = sorted(fitness_new_arr, key=lambda x: x[0], reverse = True) # sorts according to first key

  fit_gen_new = np.array([t[0] for t in fitness_new_arr])
  #print(len(fitness_arr),np.mean(fit[:20]))
  avg_new_fitness_pop=np.mean(fit_gen_new[:20])
  return buy_tree_pop_new,sell_tree_pop_new,avg_new_fitness_pop

##WarmStart to increase tree depth


# WarmStart Version 2

In [99]:
def addDepth_Binary(buys,n):
    h = buys[0].height

    cnt = len(buys) #100
    tot = (int)(0.5*cnt*(cnt-1)) #4950
    #print("total, n: ",tot,n)
    arr = random.sample(range(1,tot),n-1)

    buys_new = []

    # prefSum
    s = 0
    ps = []
    c = cnt - 1
    while c > 0:
      s+=c
      ps.append(s)
      c-=1


    for i in arr:

      left_id = bisect.bisect_right(ps, i)
      #print("left_id:",left_id)
      rr = i - ps[max(left_id-1,0)]
      right_id = left_id + rr
      #print("right_id:",right_id)
      op_id = np.random.randint(0,2)
      root = TreeNode(val = op_id)
      root.left = buys[left_id]
      root.right = buys[right_id]
      root.height = h+1

      buys_new.append(root)

    return buys_new

def addDepth_Unary(buys,n):
  arr = random.sample(range(0,len(buys)),n)
  buys_new = []
  for i in arr:
      root = TreeNode(val = 2)
      root.left = buys[i]
      root.height = root.left.height + 1
      buys_new.append(root)
  return buys_new

In [100]:
def warmStartInit(p,k,buy_pop):

    cnt = len(buy_pop) #100
    tot = (int)(0.5*cnt*(cnt-1)) #4950

    bu = np.random.randint(0,len(buy_pop)) # number of unary signals
    bb = min(p*k-bu,tot) # number of binary signals
    binarySignals = addDepth_Binary(buy_pop,bb)
    unarySignals = addDepth_Unary(buy_pop,bu)

    return binarySignals + unarySignals

In [101]:
def warmStart(dataset,base_buy_arr,base_sell_arr,buys,sells,n):
  buys = warmStartInit(n,2,buys)
  sells = warmStartInit(n,2,sells)
  fitness_arr = [(chromosome_fitness_check(base_buy_arr,base_sell_arr,buys[i],sells[i],dataset),i) for i in range(len(buys))]
  #fitness_arr = sorted(fitness_arr, key=lambda x: x[0], reverse = True) # sorts according to first key
  rnd_smp=random.sample(range(len(fitness_arr)),n)
  fitness_arr = [fitness_arr[i] for i in rnd_smp]
  #fitness_arr = fitness_arr[:n] # selects top n elements

  buys_new=[buys[t[1]] for t in fitness_arr]
  sells_new=[sells[t[1]] for t in fitness_arr]
  # buys_new_ans=np.array(buys_new)
  # sells_new_ans=np.array(sells_new)
  fitness_arr_ans=np.array([t[0] for t in fitness_arr])
  fit = np.mean(fitness_arr_ans)
  return buys_new, sells_new, fit

#Find the Best Strategy at each depth

In [102]:

def bestStrategy_at_depth_d(dataset,base_buy_arr,base_sell_arr,buys,sells,n = 50,iterations = 50,stopping_it = 5):
  '''
  input:
  p: population size
  buy_pop: base buys strategies of depth d-1
  sell_pop: base sells strategies of depth d-1
  output:
  best_buy_pop: buy signals of last generation
  best_sell_pop: sell signals of last generation
  best_fit: fitness of best population of iteration
  fit_arr: avg_fitness array of all generations{will be used to plot}
  '''
  epsla = 0.001
  buy_pop, sell_pop, fit = warmStart(dataset,base_buy_arr,base_sell_arr,buys,sells,n)
  fit_arr = []
  best_fit,prev_fit,cnt,best_Fit_id= 0,0,1,-1
  buy_opt = buy_pop
  sell_opt = sell_pop
  for it in range(iterations):
    print(f"generation {it} started!!")

    buy_pop, sell_pop, next_fit = next_generation(base_buy_arr,base_sell_arr,buy_pop,sell_pop,dataset)

    # print(f"generation {it} gone!!")
    fit_arr.append(next_fit)
    if next_fit > best_fit:
      best_fit = next_fit
      buy_opt = buy_pop
      sell_opt = sell_pop
      best_Fit_id = it
    if abs(next_fit - prev_fit) <= epsla:
      if cnt == stopping_it:
        break
      cnt += 1
      # print("cnt is:",cnt)
    else:
      cnt = 1
    prev_fit = next_fit
  fit_arr = np.array(fit_arr)
  return buy_opt,sell_opt,best_fit,fit_arr,best_Fit_id

##Finally Integrate the above steps


In [9]:
def printTree(root):
    def getHeight(root):
        if root is None:
            return 0
        return 1 + max(getHeight(root.left), getHeight(root.right))

    def fill(res, root, i, l, r):
        if root is None:
            return
        mid = (l + r) // 2
        res[i][mid] = str(root.val)
        # + ',' + str(root.ismut)
        fill(res, root.left, i + 1, l, mid - 1)
        fill(res, root.right, i + 1, mid + 1, r)

    height = getHeight(root)
    width = (1 << height) - 1  # Full binary tree width
    res = [[" " for _ in range(width)] for _ in range(height)]
    fill(res, root, 0, 0, width - 1)

    for line in res:
        print("".join(line))


In [104]:
# root = TreeNode(1)
# root.left = TreeNode(2)
# root.right = TreeNode(3)
# root.left.left = TreeNode(4)
# root.left.right = TreeNode(5)

# printTree(root)

   1   
 2   3 
4 5    


In [105]:
def integrator(dataset,base_buy_arr,base_sell_arr,baseBuys,baseSells,depth = 3):
  buy_opt = baseBuys
  sell_opt = baseSells
  dicti = {}

  for d in range(2,depth+1):
    # printTree(buy_opt[0])
    # printTree(sell_opt[0])

    buy_opt,sell_opt,best_fit,fit_arr,best_Fit_id = bestStrategy_at_depth_d(dataset,base_buy_arr,base_sell_arr,buy_opt,sell_opt)

    print("DEPTH AND BEST_FITNESS of POPULATION AND ITER: ", d,best_fit, best_Fit_id)
    dicti[d] = buy_opt,sell_opt,best_fit,fit_arr
  return dicti

# Result

In [106]:
import pandas as pd
file_path = '/content/drive/MyDrive/AllFilters.csv'
df = pd.read_csv(file_path)
print(df.shape)
df.head()

(3125, 33)


,Close,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32
0,1.296008,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.294096,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.305602,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.293142,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1.279607,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
x = 16
baseBuyTrees = [TreeNode(i) for i in range(x)]
baseSellTrees = [TreeNode(i) for i in range(x)]

base_fit_buy_arr = (df[df.columns[1:1+x]].values).T
base_fit_sell_arr = (df[df.columns[1+x:1+2*x]].values).T
dataset = df['Close'].values

In [108]:
print(base_fit_buy_arr.shape)
base_fit_sell_arr.shape

(16, 3125)


(16, 3125)

In [110]:
dicti = integrator(dataset,base_fit_buy_arr,base_fit_sell_arr,baseBuyTrees,baseSellTrees,depth = 5)

generation 0 started!!


/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


50 0.1669129058290638
generation 1 started!!
50 0.2670972158617671
generation 2 started!!
50 0.2289312783641332
generation 3 started!!
50 0.2023719376108762
generation 4 started!!
50 0.19307778958356722
generation 5 started!!
50 0.2652151181730006
generation 6 started!!
50 0.27308934938945806
generation 7 started!!
50 0.3449605724339729
generation 8 started!!
50 0.3458331452491484
generation 9 started!!
50 0.2946822484085191
generation 10 started!!
50 0.34283168383565565
generation 11 started!!
50 0.22109083869939608
generation 12 started!!
50 0.3254954707037513
generation 13 started!!
50 0.3370930702582445
generation 14 started!!
50 0.2681716557852587
generation 15 started!!
50 0.3544212049105018
generation 16 started!!
50 0.30126592956431025
generation 17 started!!
50 0.3398385355756484
generation 18 started!!
50 0.3266879130298176
generation 19 started!!
50 0.33392340999990994
generation 20 started!!
50 0.309409420787661
generation 21 started!!
50 0.32699874704006227
generation 22 s

In [111]:
import pickle


# Saving the dictionary to a pickle file
with open('/content/drive/My Drive/result_metadata_sharedfitness_run2.pkl', 'wb') as pickle_file:
    pickle.dump(dicti, pickle_file)

In [6]:
# Loading the dictionary from the pickle file
import pickle
with open('/content/drive/My Drive/result_metadata_sharedfitness_run2.pkl', 'rb') as pickle_file:
    loaded_dicti = pickle.load(pickle_file)

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
import math
for d in range(2,6):
  buy_opt,sell_opt,best_fit,fit_arr = loaded_dicti[d]
  print(f"Avg fitness(sharpe) of top 20 individuals of population at depth {d}:", math.sqrt(3125)*best_fit)
  i = 1
  for buy,sell in zip(buy_opt,sell_opt):
    print(f"strategy {i} ->")
    print("buy rule: ")
    printTree(buy)
    print("sell rule: ")
    printTree(sell)
    i += 1



Avg fitness(sharpe) of top 20 individuals of population at depth 2: 23.36316745750326
strategy 1 ->
buy rule: 
 1 
8 10
sell rule: 
 1 
2 7
strategy 2 ->
buy rule: 
 1 
13 11
sell rule: 
 1 
14 7
strategy 3 ->
buy rule: 
 1 
13 11
sell rule: 
 1 
14 7
strategy 4 ->
buy rule: 
 1 
13 13
sell rule: 
 1 
6 12
strategy 5 ->
buy rule: 
 1 
13 11
sell rule: 
 1 
14 7
strategy 6 ->
buy rule: 
 0 
13 13
sell rule: 
 1 
6 12
strategy 7 ->
buy rule: 
 0 
8 11
sell rule: 
 1 
2 7
strategy 8 ->
buy rule: 
 1 
13 11
sell rule: 
 1 
14 7
strategy 9 ->
buy rule: 
 1 
13 11
sell rule: 
 1 
14 7
strategy 10 ->
buy rule: 
 1 
13 11
sell rule: 
 0 
14 7
strategy 11 ->
buy rule: 
 1 
8 10
sell rule: 
 1 
2 7
strategy 12 ->
buy rule: 
 1 
13 11
sell rule: 
 1 
14 7
strategy 13 ->
buy rule: 
 1 
13 13
sell rule: 
 1 
6 12
strategy 14 ->
buy rule: 
 1 
13 11
sell rule: 
 1 
14 7
strategy 15 ->
buy rule: 
 1 
8 10
sell rule: 
 1 
2 7
strategy 16 ->
buy rule: 
 1 
13 11
sell rule: 
 1 
6 7
strategy 17 ->
buy r